<a href="https://colab.research.google.com/github/izu-hiro/RAG-Pageindex/blob/main/prototipo_rag/quary.teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Definindo a LLM

In [20]:
from openai import AsyncOpenAI
from getpass import getpass

openrouterKey = getpass("Insira a sua chave API: ")

async def call_llm(prompt, model="deepseek/deepseek-chat-v3.1:free", temperature=0):
    client = AsyncOpenAI(base_url="https://openrouter.ai/api/v1", api_key=openrouterKey)
    response = await client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

Conectando com o PageIndex

In [21]:
from pageindex import PageIndexClient

pi_client = PageIndexClient(api_key=getpass("Insira a chave API do Page Index: "))

Contexto utilizado na geração da resposta

In [22]:
from pypdf import PdfWriter
from os import listdir
from os.path import isfile, join

documentos = ['documentos/'+f for f in listdir('documentos') if isfile(join('documentos', f))]

merger = PdfWriter()
for documento in documentos:
    merger.append(documento)
merger.write("documentoUnido.pdf")
merger.close()



Enviar arquivo e pegar o ID

In [29]:
result = pi_client.submit_document("./documentoUnido.pdf")
doc_id = result["doc_id"]

# Get OCR results in page format (default)
ocr_result = pi_client.get_ocr(doc_id)
if ocr_result.get("status") == "completed":
    print("OCR Results:", ocr_result.get("result"))

'''
# Get OCR results in node format
ocr_result = pi_client.get_ocr(doc_id, format="node")
if ocr_result.get("status") == "completed":
    print("OCR Results:", ocr_result.get("result"))

# Get OCR results in raw format (concatenated markdown)
ocr_result = pi_client.get_ocr(doc_id, format="raw")
if ocr_result.get("status") == "completed":
    print("Raw Markdown:", ocr_result.get("result"))
'''

tree_result = pi_client.get_tree(doc_id)
if tree_result.get("status") == "completed":
    print("PageIndex Tree Structure:", tree_result.get("result"))

KeyboardInterrupt: 

Retrieval

In [15]:
from time import sleep
def retrieval(query):
    while True:
        if pi_client.is_retrieval_ready(doc_id):
            retrieval = pi_client.submit_query(doc_id, query)
            retrieval_id = retrieval['retrieval_id']
            break
        else:
            print("Document is not ready for retrieval yet")
            sleep(5)

    while True:
        retrieval_result = pi_client.get_retrieval(retrieval_id)
        if retrieval_result.get("status") == "completed":
            return retrieval_result.get('result')
        sleep(1)


Geração da resposta

In [19]:
print("### INICIANDO DEMONSTRAÇÃO ###\n")

# Pergunta original e específica do usuário
original_query = "Se eu me inscrever no último dia, 07/11/2025, e pagar a taxa com cartão de crédito, minha inscrição para o curso de Radiologia, que tem requisito de idade, será confirmada imediatamente?"
print(f"Pergunta Original Específica:\n{original_query}\n")


# prompt pra llm

step_back_prompt = f"""
Dada a seguinte pergunta específica, formule uma pergunta "step-back" mais geral que capture os conceitos fundamentais necessários para respondê-la.

Pergunta Específica: "{original_query}"

Pergunta Step-Back Geral:
"""

print("--- PASSO 1: Gerando a Pergunta ---")

step_back_question = await call_llm(step_back_prompt)
print(f"Pergunta Gerada: {step_back_question}\n")


print("--- PASSO 2: Utilizando o contexto já carregado ---")
retrieved_context = retrieval(step_back_question)
print("Contexto recuperado com sucesso.\n")



# prompt final que orienta o LLM a usar o contexto e a pergunta geral
final_answer_prompt = f"""
Você deve responder à "Pergunta Específica" usando o "Contexto" fornecido.
Para te guiar, primeiro considere a "Pergunta Step-Back Geral" e como o contexto a responde.
Use esse entendimento para construir uma resposta completa e precisa para a pergunta original.

Contexto:
---
{retrieved_context}
---

Pergunta Step-Back Geral: {step_back_question}
Pergunta Específica: {original_query}

Resposta Final Detalhada:
"""

print("--- PASSO 3: Gerando a Resposta Final ---")

final_answer = await call_llm(final_answer_prompt)
print("Resposta Gerada:\n")
print(final_answer)


### INICIANDO DEMONSTRAÇÃO ###

Pergunta Original Específica:
Se eu me inscrever no último dia, 07/11/2025, e pagar a taxa com cartão de crédito, minha inscrição para o curso de Radiologia, que tem requisito de idade, será confirmada imediatamente?

--- PASSO 1: Gerando a Pergunta ---


NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy (Free model publication). Configure: https://openrouter.ai/settings/privacy', 'code': 404}}